In [ ]:
import pandas as pd
btc = pd.read_csv('/content/15MIN (1).csv')
btc.head()

,time_period_start,time_period_end,price_open,price_high,price_low,price_close,volume_traded
0,2021-01-30 17:30:00,2021-01-30 17:45:00,34190.25,34239.84,34056.18,34112.00,17.526346
1,2021-01-30 17:45:00,2021-01-30 18:00:00,34122.68,34279.45,34095.70,34237.36,21.251899
2,2021-01-30 18:00:00,2021-01-30 18:15:00,34258.75,34335.37,34107.93,34223.35,46.654135
3,2021-01-30 18:15:00,2021-01-30 18:30:00,34229.93,34363.25,34216.94,34291.82,27.195698
4,2021-01-30 18:30:00,2021-01-30 18:45:00,34297.86,34306.84,34139.57,34180.00,21.800071


In [ ]:
btc = btc.drop(['time_period_end'], axis=1)
btc = btc.rename(columns={"time_period_start": "timestamp"})
btc['timestamp'] = pd.to_datetime(btc['timestamp'], format='%Y-%m-%d %H:%M:%S')
btc['timestamp'] = btc['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

btc.info

<bound method DataFrame.info of                  timestamp  price_open  price_high  price_low  price_close  \
0      2021-01-30 17:30:00    34190.25    34239.84   34056.18     34112.00   
1      2021-01-30 17:45:00    34122.68    34279.45   34095.70     34237.36   
2      2021-01-30 18:00:00    34258.75    34335.37   34107.93     34223.35   
3      2021-01-30 18:15:00    34229.93    34363.25   34216.94     34291.82   
4      2021-01-30 18:30:00    34297.86    34306.84   34139.57     34180.00   
...                    ...         ...         ...        ...          ...   
32181  2021-12-31 22:45:00    46281.00    46462.67   46254.81     46322.70   
32182  2021-12-31 23:00:00    46322.70    46467.04   46313.24     46437.73   
32183  2021-12-31 23:15:00    46450.59    46479.19   46345.18     46345.18   
32184  2021-12-31 23:30:00    46371.97    46488.57   46371.97     46483.24   
32185  2021-12-31 23:45:00    46434.34    46435.77   46140.00     46214.37   

       volume_traded  
0       

# Calculate and Append MACD

In [ ]:
def calculate_SMA12(head):
    return head['price_close'].sum()/12

def calculate_SMA26(head):
    return head['price_close'].sum()/26

def calculate_EMA12(df):
    ema12_list = []
    prev_ema = -1
    k = 2/(12+1)
    for ind in range(len(df)):
        if ind < 26:
            ema12_list.append(-1)
        else:
            if prev_ema == -1:
                sma = calculate_SMA12(df[14:ind])
                ema = k*(df['price_close'][ind]-sma)+sma
                prev_ema = ema
            else:
                ema = k*(df['price_close'][ind]-prev_ema)+prev_ema
            ema12_list.append(ema)

    return ema12_list

def calculate_EMA26(df):
    ema26_list = []
    prev_ema = -1
    k = 2/(26+1)
    for ind in range(len(df)):
        if ind < 26:
            ema26_list.append(-1)
        else:
            if prev_ema == -1:
                sma = calculate_SMA26(df[0:ind])
                ema = k*(df['price_close'][ind]-sma)+sma
                prev_ema = ema
            else:
                ema = k*(df['price_close'][ind]-prev_ema)+prev_ema
            ema26_list.append(ema)

    return ema26_list

def calculate_and_append_MACD(df):
    macd_list = []
    ema12_list = calculate_EMA12(df)
    ema26_list = calculate_EMA26(df)
    for i in range(len(ema12_list)):
        macd_list.append(ema12_list[i] - ema26_list[i])
    df['MACD'] = macd_list

    return df

In [ ]:
btc = calculate_and_append_MACD(btc)

btc.head

<bound method NDFrame.head of                  timestamp  price_open  price_high  price_low  price_close  \
0      2021-01-30 17:30:00    34190.25    34239.84   34056.18     34112.00   
1      2021-01-30 17:45:00    34122.68    34279.45   34095.70     34237.36   
2      2021-01-30 18:00:00    34258.75    34335.37   34107.93     34223.35   
3      2021-01-30 18:15:00    34229.93    34363.25   34216.94     34291.82   
4      2021-01-30 18:30:00    34297.86    34306.84   34139.57     34180.00   
...                    ...         ...         ...        ...          ...   
32181  2021-12-31 22:45:00    46281.00    46462.67   46254.81     46322.70   
32182  2021-12-31 23:00:00    46322.70    46467.04   46313.24     46437.73   
32183  2021-12-31 23:15:00    46450.59    46479.19   46345.18     46345.18   
32184  2021-12-31 23:30:00    46371.97    46488.57   46371.97     46483.24   
32185  2021-12-31 23:45:00    46434.34    46435.77   46140.00     46214.37   

       volume_traded         MACD

# SMA

In [ ]:
def calculate_SMA(head):
    return head['price_close'].sum()/5

def calculate_and_append_SMA(df):
    sma_list = []
    for ind in range(len(df)):
        if ind < 26:
            sma_list.append(-1)
        else:
            sma = calculate_SMA(df[ind-4:ind+1])
            sma_list.append(sma)
    df['SMA'] = sma_list

    return df

In [ ]:
btc = calculate_and_append_SMA(btc)

btc.head

<bound method NDFrame.head of                  timestamp  price_open  price_high  price_low  price_close  \
0      2021-01-30 17:30:00    34190.25    34239.84   34056.18     34112.00   
1      2021-01-30 17:45:00    34122.68    34279.45   34095.70     34237.36   
2      2021-01-30 18:00:00    34258.75    34335.37   34107.93     34223.35   
3      2021-01-30 18:15:00    34229.93    34363.25   34216.94     34291.82   
4      2021-01-30 18:30:00    34297.86    34306.84   34139.57     34180.00   
...                    ...         ...         ...        ...          ...   
32181  2021-12-31 22:45:00    46281.00    46462.67   46254.81     46322.70   
32182  2021-12-31 23:00:00    46322.70    46467.04   46313.24     46437.73   
32183  2021-12-31 23:15:00    46450.59    46479.19   46345.18     46345.18   
32184  2021-12-31 23:30:00    46371.97    46488.57   46371.97     46483.24   
32185  2021-12-31 23:45:00    46434.34    46435.77   46140.00     46214.37   

       volume_traded         MACD

# OBV

In [ ]:
def calculate_and_append_OBV(df):
    obv_list = []
    obv = 0
    for ind in range(len(df)):
        if ind < 26:
            obv_list.append(-1)
        else:
            if df['price_close'][ind] > df['price_close'][ind-1]:
                obv = obv + df['volume_traded'][ind]
            elif df['price_close'][ind] < df['price_close'][ind-1]:
                obv = obv - df['volume_traded'][ind]
            obv_list.append(obv)
    df['OBV'] = obv_list

    return df

In [ ]:
btc = calculate_and_append_OBV(btc)
btc.info

<bound method DataFrame.info of                  timestamp  price_open  price_high  price_low  price_close  \
0      2021-01-30 17:30:00    34190.25    34239.84   34056.18     34112.00   
1      2021-01-30 17:45:00    34122.68    34279.45   34095.70     34237.36   
2      2021-01-30 18:00:00    34258.75    34335.37   34107.93     34223.35   
3      2021-01-30 18:15:00    34229.93    34363.25   34216.94     34291.82   
4      2021-01-30 18:30:00    34297.86    34306.84   34139.57     34180.00   
...                    ...         ...         ...        ...          ...   
32181  2021-12-31 22:45:00    46281.00    46462.67   46254.81     46322.70   
32182  2021-12-31 23:00:00    46322.70    46467.04   46313.24     46437.73   
32183  2021-12-31 23:15:00    46450.59    46479.19   46345.18     46345.18   
32184  2021-12-31 23:30:00    46371.97    46488.57   46371.97     46483.24   
32185  2021-12-31 23:45:00    46434.34    46435.77   46140.00     46214.37   

       volume_traded         MA

# RSI

In [ ]:
def init_rsi(df):
    gain = []
    loss = []
    tmp = 0
    for ind in range(13, 26):
        diff = df['price_close'][ind] - df['price_close'][ind-1]
        if diff > 0:
            gain.append(diff)
        else:
            loss.append(-1*diff)
    avg_gain = sum(gain)/14
    avg_loss = sum(loss)/14

    return avg_gain, avg_loss

def calculate_and_append_RSI(df):
    rsi_list = []
    avg_gain = 0
    avg_loss = 0
    for ind in range(len(df)):
        if ind < 26:
            rsi_list.append(-1)
        elif ind == 26:
            avg_gain, avg_loss = init_rsi(df[12:ind])
            rsi = 100 - 100/(1 + avg_gain/avg_loss)
            rsi_list.append(rsi)
        else:
            diff = df['price_close'][ind] - df['price_close'][ind-1]
            if diff > 0:
                avg_gain = (avg_gain*13 + diff) / 14
            elif diff < 0:
                avg_loss = (avg_loss*13 + -1*diff) / 14
            rsi = 100 - 100/(1 + avg_gain/avg_loss)
            rsi_list.append(rsi)

    df['RSI'] = rsi_list

    return df


In [ ]:
btc = calculate_and_append_RSI(btc)

btc.info

<bound method DataFrame.info of                  timestamp  price_open  price_high  price_low  price_close  \
0      2021-01-30 17:30:00    34190.25    34239.84   34056.18     34112.00   
1      2021-01-30 17:45:00    34122.68    34279.45   34095.70     34237.36   
2      2021-01-30 18:00:00    34258.75    34335.37   34107.93     34223.35   
3      2021-01-30 18:15:00    34229.93    34363.25   34216.94     34291.82   
4      2021-01-30 18:30:00    34297.86    34306.84   34139.57     34180.00   
...                    ...         ...         ...        ...          ...   
32181  2021-12-31 22:45:00    46281.00    46462.67   46254.81     46322.70   
32182  2021-12-31 23:00:00    46322.70    46467.04   46313.24     46437.73   
32183  2021-12-31 23:15:00    46450.59    46479.19   46345.18     46345.18   
32184  2021-12-31 23:30:00    46371.97    46488.57   46371.97     46483.24   
32185  2021-12-31 23:45:00    46434.34    46435.77   46140.00     46214.37   

       volume_traded         MA

# MFI

In [ ]:
def calculate_money_ratio(df, start, end):
    pos_flow_14 = 0
    neg_flow_14 = 0
    prev_typical_price = (df['price_low'][start]+df['price_high'][start]+df['price_close'][start])/3
    

    for ind in range(start, end):
        typical_price = (df['price_low'][ind]+df['price_high'][ind]+df['price_close'][ind])/3
        if typical_price > prev_typical_price:
            pos_flow_14 = pos_flow_14 + typical_price*df['volume_traded'][ind]
        else:
            neg_flow_14 = neg_flow_14 + typical_price*df['volume_traded'][ind]
    money_ratio = pos_flow_14/neg_flow_14

    return money_ratio

def calculate_and_append_MFI(df):
    mfi_list = []
    for ind in range(len(df)):
        if ind < 26:
            mfi_list.append(-1)
        else:           
            mr = calculate_money_ratio(df[ind-13:ind+1], ind-13, ind+1)
            mfi = 100 - 100/(1+mr)
            mfi_list.append(mfi)
    df['MFI'] = mfi_list

    return df

In [ ]:
btc = calculate_and_append_MFI(btc)

btc.info

<bound method DataFrame.info of                  timestamp  price_open  price_high  price_low  price_close  \
0      2021-01-30 17:30:00    34190.25    34239.84   34056.18     34112.00   
1      2021-01-30 17:45:00    34122.68    34279.45   34095.70     34237.36   
2      2021-01-30 18:00:00    34258.75    34335.37   34107.93     34223.35   
3      2021-01-30 18:15:00    34229.93    34363.25   34216.94     34291.82   
4      2021-01-30 18:30:00    34297.86    34306.84   34139.57     34180.00   
...                    ...         ...         ...        ...          ...   
32181  2021-12-31 22:45:00    46281.00    46462.67   46254.81     46322.70   
32182  2021-12-31 23:00:00    46322.70    46467.04   46313.24     46437.73   
32183  2021-12-31 23:15:00    46450.59    46479.19   46345.18     46345.18   
32184  2021-12-31 23:30:00    46371.97    46488.57   46371.97     46483.24   
32185  2021-12-31 23:45:00    46434.34    46435.77   46140.00     46214.37   

       volume_traded         MA

In [ ]:
btc.to_csv('/content/15MIN_Tech.csv', index=False)